# Autobus - Live demo

In [1]:
import time
print(time.ctime(),'go')

#####Make Robot#####
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
robot = Robot()

#####UI functions#####
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display

#####Monitor Network Connection#####
from jetbot import Heartbeat
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        print('check network connection')
heartbeat = Heartbeat(period=5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status

Mon Aug  3 00:26:29 2020 go


### Rail Tracking Setup

In [2]:
#####Rail Tracking Setup#####
import torchvision
import torch
steer_model_file = 'best_steering_model_xy--PoolTable400.pth'
steer_model = torchvision.models.resnet18(pretrained=False)
steer_model.fc = torch.nn.Linear(512, 2)
steer_model.load_state_dict(torch.load(steer_model_file))
steer_device = torch.device('cuda')
steer_model = steer_model.to(steer_device)
steer_model = steer_model.eval().half()
print(time.ctime(),steer_model_file,'ok')

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
steer_mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
steer_std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
def steer_preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(steer_device).half()
    image.sub_(steer_mean[:, None, None]).div_(steer_std[:, None, None])
    return image[None, ...]

#add the steering xy prediction to the image
def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

print(time.ctime(),'ok')

Mon Aug  3 00:26:47 2020 best_steering_model_xy--PoolTable400.pth ok
Mon Aug  3 00:26:47 2020 ok


### Collision Avoidance Setup

In [3]:
#####Collision Avoidance Setup#####
import torch
import torchvision

oba_model = torchvision.models.alexnet(pretrained=False)
oba_model.classifier[6] = torch.nn.Linear(oba_model.classifier[6].in_features, 2)
#oba_model.load_state_dict(torch.load('best_model.pth'))
oba_model.load_state_dict(torch.load('best_obstacle_model--NVIDIA.pth'))
oba_device = torch.device('cuda')
oba_model = oba_model.to(oba_device)
print(time.ctime(),'oba_model','ok')

import cv2
import numpy as np

oba_mean = 255.0 * np.array([0.485, 0.456, 0.406])
oba_stdev = 255.0 * np.array([0.229, 0.224, 0.225])
oba_normalize = torchvision.transforms.Normalize(oba_mean, oba_stdev)
def oba_preprocess(camera_value):
    global oba_device, oba_normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = oba_normalize(x)
    x = x.to(oba_device)
    x = x[None, ...]
    return x

import torch.nn.functional as F
import time

print(time.ctime(),'ok')

check network connection
Mon Aug  3 00:26:51 2020 oba_model ok
Mon Aug  3 00:26:51 2020 ok


### Create a function that will get called whenever the camera's value changes. 

In [4]:
##### Create a function that will get called whenever the camera's value changes. #####
#This function will do the following steps
#- Pre-process the camera image
#- Execute the neural network
#- Stop and prevent motion on obstacle detection
#- Stop when jetbot reaches a station, resumes after a set time
#- If OK to move, control steering, keep jetbot on the track

#####Global vars for execute#####
#all vars can be changed later, even while AUTOBUS is running
angle = 0.0
angle_last = 0.0
mt = 0
frame_cnt = 0
interval = 0.1
stutter = 0
steer_debug = False
tt = time.time()
fwd_high = 0.4
fwd_low = 0.3
turn_high = 0.4
turn_low = 0.3
stutter_spd = 0.1
prob_block_stop = 0.8
time_last_station = 9000 #always start code with station stopping enabled
leave_station_secs = 60

#!!!!! Jetbot doing something with new image !!!!!
def execute(change):
    global angle, angle_last, mt, frame_cnt, tt, steer_debug, interval, accel, stutter, robot
    global stutter_spd, prob_block_stop, min_inter, max_inter, fwd_high, fwd_low, turn_high, turn_low
    
    #count images read
    frame_cnt += 1
    
    #min time interval between each run
    if time.time() < tt + interval:
        return
    tt = time.time()
    
    #stutters robot, start-stop motion
    if stutter:
        rlmv = robot.left_motor.value 
        rrmv = robot.right_motor.value
        robot.left_motor.value = stutter_spd *mt
        robot.right_motor.value = stutter_spd *mt
        time.sleep(0.1) #stop to get clear image
        robot.left_motor.value = rlmv
        robot.right_motor.value = rrmv
        
    #AAAAA jetbot reads image and makes a obstacle stop decision AAAAA
    x = change['new'] 
    x = oba_preprocess(x)
    y = oba_model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    status_b.value = str(frame_cnt) + " obstacle prob: " + "{:.2f}".format(prob_blocked)
    if prob_blocked > prob_block_stop:
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        status_a.value = str(frame_cnt) + " obstacle stop"
        return
    #^^^^^ jetbot finishes obstacle stop decision ^^^^^
        
    #BBBBB jetbot reads image and makes a station stop decision BBBBB
    """
        Purpose:
            Detects color of camera frame image.

        Args:
            None.

        Returns:
            color_detected: string value of either 'red', 'yellow', 'green', 'purple' or 'other'

        Raises:
            ConnectionError: If no available port is found.
        
        References:
            (1) To understand colors and the color space with OpenCV read this link:
                - https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/
            (2) To see color detection in OpenCV read this link:
                - https://www.learnopencv.com/invisibility-cloak-using-color-detection-and-segmentation-with-opencv/
            (3) To read more on smoothing images read this link:
                - https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_filtering/py_filtering.html
            (4) To read more on morphological transformations read this link:
                - https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
            (5) To choose upper and lower limits manually follow this link:
                - https://toolstud.io/color/rgb.php
            (6) To understand multiple color detection in real-time using OpenCV read this link:
                - https://www.geeksforgeeks.org/multiple-color-detection-in-real-time-using-python-opencv/
    """
    
    image_frame = change['new']
    
    # extracting out HSV color space values:
    # H – Hue (Dominant Wavelength)
    # S – Saturation (Purity/Shades of Color)
    # V – Value (Intensity)
    hsv = cv2.cvtColor(image_frame, cv2.COLOR_BGR2HSV)
    
    # adding median blur to image
    hsv = cv2.medianBlur(hsv,5)
    
    # lower range of red
    bttm_lower_red = np.array([0,120,70])
    top_lower_red = np.array([10,255,255])
    lower_red_mask = cv2.inRange(hsv, bttm_lower_red, top_lower_red)

    # upper range of red
    bttm_upper_red = np.array([170,120,70])
    top_upper_red = np.array([180,255,255])
    upper_red_mask = cv2.inRange(hsv, bttm_upper_red, top_upper_red)

    # generating the final mask to detect red color
    red_range_mask = lower_red_mask + upper_red_mask

    # removing noise through opening (erosion followed by dilation)
    red_range_mask = cv2.morphologyEx(red_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

    # increasing white region or size of foreground object increases through dilation
    red_range_mask = cv2.morphologyEx(red_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

    # detecting if image frame contains red
    red_results = cv2.bitwise_and(image_frame, image_frame, mask = red_range_mask)

    # calculating ratio of red in image frame and prints output
    red_ratio = (cv2.countNonZero(red_range_mask))/(image_frame.size/3)
    ###print("Amount of red in image:", np.round(red_ratio*100, 2))
    
    # generating mask for range of yellow
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    yellow_range_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # removing noise through opening (erosion followed by dilation)
    yellow_range_mask = cv2.morphologyEx(yellow_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

    # increasing white region or size of foreground object increases through dilation
    yellow_range_mask = cv2.morphologyEx(yellow_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

    # detecting if image frame contains yellow
    yellow_results = cv2.bitwise_and(image_frame, image_frame, mask = yellow_range_mask)

    # calculating ratio of yellow in image frame and prints output
    yellow_ratio = (cv2.countNonZero(yellow_range_mask))/(image_frame.size/3)
    ###print("Amount of yellow in image:", np.round(yellow_ratio*100, 2))
    
    # generating mask for range of green
    lower_green = np.array([36, 25, 25])
    upper_green = np.array([86, 255,255])
    green_range_mask = cv2.inRange(hsv, lower_green, upper_green)

    # removing noise through opening (erosion followed by dilation)
    green_range_mask = cv2.morphologyEx(green_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

    # increasing white region or size of foreground object increases through dilation
    green_range_mask = cv2.morphologyEx(green_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

    # detecting if image frame contains green
    green_results = cv2.bitwise_and(image_frame, image_frame, mask = green_range_mask)

    # calculating ratio of green in image frame and prints output
    green_ratio = (cv2.countNonZero(green_range_mask))/(image_frame.size/3)
    ###print("Amount of green in image:", np.round(green_ratio*100, 2))
    
    # generating mask for range of purple
    lower_purple = np.array([80, 10, 10])
    upper_purple = np.array([120, 255, 255])
    purple_range_mask = cv2.inRange(hsv, lower_purple, upper_purple)

    # removing noise through opening (erosion followed by dilation)
    purple_range_mask = cv2.morphologyEx(purple_range_mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))

    # increasing white region or size of foreground object increases through dilation
    purple_range_mask = cv2.morphologyEx(purple_range_mask, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))

    # detecting if image frame contains purple
    purple_results = cv2.bitwise_and(image_frame, image_frame, mask = purple_range_mask)

    # calculating ratio of purple in image frame and prints output
    purple_ratio = (cv2.countNonZero(purple_range_mask))/(image_frame.size/3)
    ###print("Amount of purple in image:", np.round(purple_ratio*100, 2))
    
    # calculating ratio of other colors in image frame and prints output
    rygp_total = (red_ratio + yellow_ratio + green_ratio + purple_ratio)
    other_ratio = (100 - rygp_total)
    ###print("Amount of other colors in image:", np.round(purple_ratio*100, 2))
    
    # assigning color detected to variable
    if red_ratio >= 5:

        # save detected color value as red
        color_detected = 'red'

    elif yellow_ratio >= 5:

        # save detected color value as yellow
        color_detected = 'yellow'

    elif green_ratio >= 5:

        # save detected color value as green
        color_detected = 'green'

    elif purple_ratio >= 23:

        # save detected color value as purple
        color_detected = 'purple'

    else:

        # save detected color value as other
        color_detected = 'other'
 
    # output deteced color value
    status_c.value = str(frame_cnt) + " color detected: " + color_detected
    
    # jetbot decides to make a station stop, or not 
    if color_detected == 'red':
        #dont't get stuck stopping in same station
        #if time since stopping at last station is more than time it takes to leave a station...
        if time.time() - time_last_station > leave_station_secs: 
            status_a.value = str(frame_cnt) + " station stop"
            robot.left_motor.value = 0
            robot.right_motor.value = 0
            for i in range(60): #stop, sleep for x secs, then keep moving
                time.sleep(1) 
                status_a.value = str(frame_cnt) + " station stop " + i + " seconds"
            time_last_station = time.time() #keep track of time since leaving station
        
    #^^^^^ jetbot made decision based on color function ^^^^^
    
    """
        Purpose:
            Provides autobus with functions associated with each color it detected.

        Args:
            color_detected: string value of either 'red', 'yellow', 'green', 'purple' or 'other'

        Returns:
            None.

        Raises:
            ConnectionError: If no available port is found.
        
        References:
            (1) To understand movement functions of Jetbot Waveshare read this link:
                - https://www.elephantjay.com/blogs/tutorial/266
    """

#     if color_detected == 'red':
        
#         # if red then autobus stops
#         print("Color Code Red: Stop detected")
#         robot.stop()
#         robot.left_motor.value = 0
#         robot.right_motor.value = 0
#         return
    
#     elif color_detected == 'yellow':
        
#         # if yellow then autobus slows down
#         print("Color Code Yellow: Speed reduction detected")
#         robot.left_motor.value = (robot.left_motor.value * 0.5)
#         robot.right_motor.value = (robot.right_motor.value  * 0.5)
#         return

#     elif color_detected == 'green':
        
#         # if green then autobus continues at normal speed
#         print("Color Code Green: Continuation at normal speed detected")
#         robot.left_motor.value = 0.3
#         robot.right_motor.value = 0.3
#         break
        
#     elif color_detected == 'purple':
        
#         # if purple then autobus stops breilfly and starts again
#         print("Color Code Purple: Bus stop detected")
#         robot.stop()
#         robot.left_motor.value = 0
#         robot.right_motor.value = 0
#         robot.sleep(2.0) # 2 seconds wait
#         robot.left_motor.value = 0.3
#         robot.right_motor.value = 0.3
#         break
    
#     elif color_detected == 'other':
#         print("No Color Code Detected")
#         break

    #CCCCC jetbot reads image and makes a steering decision CCCCC
    
    image = change['new']
    xy = steer_model(steer_preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value
   
    #nnnnn use default NVIDA algo nnnnn
    if not steer_debug:
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) *mt
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) *mt
        status_a.value = str(frame_cnt) + " move"
        return
        
    #xxxxx use AUTOBUS debugging algo xxxxx
    else: 
        stutter = 1

        left_motor = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) 
        right_motor = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) 
        left_slider.value = left_motor
        right_slider.value = right_motor

        #run at preset speeds
        thres = turn_thres.value #steering value must exceed threshold to initate turning
        if steering_slider.value > thres:
            status_a.value = str(frame_cnt) + " turn right"
            robot.left_motor.value = min(robot.left_motor.value + 0.1, turn_high) *mt
            robot.right_motor.value = max(robot.right_motor.value - 0.1, turn_low) *mt
            stutter_spd = max(stutter_spd - 0.02, turn_low*0.9)
            
        elif steering_slider.value < -thres:
            status_a.value = str(frame_cnt) + " turn left"
            robot.left_motor.value = max(robot.left_motor.value - 0.1, turn_low) *mt
            robot.right_motor.value = min(robot.right_motor.value + 0.1, turn_high) *mt
            stutter_spd = max(stutter_spd - 0.02, turn_low*0.9)

        else: 
            status_a.value = str(frame_cnt) + " go straight"
            if robot.left_motor.value != robot.right_motor.value:
                robot.left_motor.value = robot.right_motor.value = min(robot.left_motor.value,robot.right_motor.value)
            #initial throttle then lower to coast speed    
            robot.left_motor.value = (robot.left_motor.value - 0.01) *mt
            robot.right_motor.value = (robot.right_motor.value - 0.01) *mt
            if min(robot.left_motor.value,robot.right_motor.value) < fwd_low:
                robot.left_motor.value = robot.right_motor.value = fwd_high *mt  
            stutter_spd = min(stutter_spd + 0.01, fwd_low*0.9)

        #set adjusted motor values shown in UI
        left_adjusted.value = robot.left_motor.value
        right_adjusted.value = robot.right_motor.value
        #^^^^^ jetbot finished steering decision ^^^^^
    
    return None
#^^^^^ jetbot image processing ^^^^^

print(time.ctime(),'ok')

Mon Aug  3 00:26:51 2020 ok


### setup UI - camera view, buttons, etc.

In [5]:
#setup interface - camera view, buttons, etc
#
#user controlled sliders
#1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
#2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
#3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets
#jetbot turns when steering value exceeds threshold (for AUTOBUS steering algo only, not in NVIDIA steering algo)
speed_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.33, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.33, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.33, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, value=0, description='steering bias')
turn_thres = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='steer thres') #turn threshold
#
#buttons
button_layout = ipywidgets.Layout(width='80px', height='35px')
run_button = ipywidgets.Button(description='START', button_style='success', layout=button_layout)
run_button.on_click(lambda x: start())
#
stop_button = ipywidgets.Button(description='STOP', button_style='danger', layout=button_layout)
stop_button.on_click(lambda x: stop())
#
camera_button = ipywidgets.Button(description='CAMERA',layout=button_layout)
camera_button.on_click(lambda x: toggle_camera())
#
motor_button = ipywidgets.Button(description='MOTOR',layout=button_layout)
motor_button.on_click(lambda x: toggle_motor())
#
stutter_button = ipywidgets.Button(description='STUTTER',layout=button_layout)
stutter_button.on_click(lambda x: toggle_stutter())
#
#read only sliders
x_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, value=0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='speed')
#
left_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='L nvidia')
right_slider = ipywidgets.FloatSlider(min=0, max=1,step=0.01, orientation='vertical', description='R nvidia')
#
left_adjusted = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='L adjusted')
right_adjusted = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='R adjusted')
#
#The x and y sliders will display the predicted x, y values.
#The steering slider will display our estimated steering value.  Please remember, this value isn't the actual 
#angle of the target, but simply a value that is nearly proportional.  When the actual angle is ``0``, this 
#will be zero, and it will increase / decrease with the actual angle.  
#
#arrange widgets in UI
status_a = ipywidgets.Textarea(value = "status a")
status_b = ipywidgets.Textarea(value = "status b")
status_c = ipywidgets.Textarea(value = "status c")
sliders = widgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,turn_thres])
buttons = widgets.VBox([run_button,stop_button,camera_button,motor_button,stutter_button])
y_sets = ipywidgets.HBox([y_slider,left_slider,right_slider,left_adjusted,right_adjusted])
x_sets = widgets.VBox([x_slider, steering_slider, ipywidgets.Label("a) action"), status_a,
                       ipywidgets.Label("b) obstacle detect"), status_b, 
                       ipywidgets.Label("c) color detect"), status_c])
#
cam_toggle = 1 #enables/disables camera, may improve performance
def toggle_camera():
    global cam_toggle
    if cam_toggle:
        camera_link.unlink()
        cam_toggle = 0
    else:
        camera_link.link()
        cam_toggle = 1
#
mt = 0 #start/stop motor
def toggle_motor():
    global mt
    if mt: mt = 0
    else: mt = 1
#       
stutter = 0 #enable/disable stutter-steps
def toggle_stutter():
    global stutter
    if stutter: stutter = 0
    else: stutter = 1
#
#start processing images
def start():
    execute({'new': camera.value})
    camera.observe(execute, names='value')
#
#stop jetbot, stop camera
def stop():
    time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
    camera.unobserve_all()
    robot.stop()
    camera.stop()
#
print(time.ctime(),'ok')

Mon Aug  3 00:26:52 2020 ok


### Display UI and Enable Controls

In [13]:
#make camera
try: camera.stop()
except: print('check camera')
fps = 10
frame_cnt = 0 #reset frame count
camera = Camera.instance(width=224, height=224, capture_width=224, capture_height=224, fps=fps)
image_widget = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=display_xy)
camera.start()
#
if not steer_debug: #hide unused controls
    turn_thres.layout.visibility = 'hidden'
    left_adjusted.layout.visibility = 'hidden'
    right_adjusted.layout.visibility = 'hidden'
#
#!!!!! adjustable vars !!!!!
#!!!!! adjustable vars !!!!!
#!!!!! adjustable vars !!!!!
mt = 0 #0 starts jetbot without moving, hit motor button to start moving
stutter = 0 #0 disable start stop stutter movement
robot.left_motor.alpha = 1 #scales motor speed by this, 1 is no change
robot.right_motor.alpha = 1 
interval = 1.0/fps #set interval between execute's same as frame rate
prob_block_stop = 0.8 #only stop if prob is higher than this
leave_station_secs = 60 #jetbot will not stop at station unless it's been x secs since it's last stop
#^^^^^ adjustable vars ^^^^^
#^^^^^ adjustable vars ^^^^^
#^^^^^ adjustable vars ^^^^^
#
#run jetbot
execute({'new': camera.value}) #test single image run
start() #start jetbot
print(time.ctime(),'ok')
#
#show UI
display(widgets.HBox([image_widget,sliders,buttons]))
display(ipywidgets.HBox([y_sets,x_sets]))
#
#IMPORTANT: The camera must be re-initialized after it stops
#Rerun this block to reinitialize camera
#First run takes a few min to start up

Mon Aug  3 00:27:37 2020 ok


///////////////////////////////////////////////End of Notebook //////////////////////////////////////////////////////////

### Debug

### Debug

In [7]:
#stop()

In [8]:
#camera.stop()

In [9]:
#robot.stop()

In [10]:
#execute({'new': camera.value})
#camera.observe(execute, names='value')

In [11]:
#start()

In [12]:
steer_debug = False #False = default NVIDIA steering; True = custom; don't use if NVIDIA steering code works fine
#????? steering debug vars ????? only in effect if steer_debug = True
if steer_debug:
    turn_high = 0.4
    turn_low = 0.3
    turn_thres.value = 0.3 #only used if steer_debug=True
    fwd_high = 0.37
    fwd_low = 0.33
    interval = 0.1
if not steer_debug: #hide not used controls
    turn_thres.layout.visibility = 'hidden'
    left_adjusted.layout.visibility = 'hidden'
    right_adjusted.layout.visibility = 'hidden'
#^^^^^ steering debug vars ^^^^^ only in effect if steer_debug = True